In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import struct

%matplotlib inline

In [ ]:
class SymbolGenerator:
    def __init__(self, data_path):
        self.data_path = data_path
        self.available_symbols = ['(', ')', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
                                 't', 'x', 'y', ',', '+', '-', '*', ' ']  # Добавили пробел
        print("Доступные символы:", self.available_symbols)
        
    def get_symbol_folder_name(self, char):
        """Преобразует символ в имя папки"""
        if char == 'x':
            return 'X'
        elif char == '*':
            return 'times'
        elif char == ' ':
            return 'space'  # Нужно создать папку для пробелов
        else:
            return char
    
    def get_random_symbol_image(self, char, train_size=0.8):
        """Получает случайное изображение символа из последних 20% данных"""
        if char == ' ':
            # Создаем пустое изображение для пробела
            return np.ones((45, 20), dtype=np.uint8) * 255
        
        folder_name = self.get_symbol_folder_name(char)
        symbol_dir = os.path.join(self.data_path, folder_name)
        
        if not os.path.exists(symbol_dir):
            print(f"Предупреждение: папка {symbol_dir} не существует")
            return np.ones((45, 45), dtype=np.uint8) * 255
        
        images = [f for f in os.listdir(symbol_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        images.sort()
        
        if not images:
            print(f"Предупреждение: нет изображений в {symbol_dir}")
            return np.ones((45, 45), dtype=np.uint8) * 255
        
        start_index = int(len(images) * train_size)
        last_20_percent = images[start_index:]
        
        if not last_20_percent:
            last_20_percent = images  # Если мало данных, берем все
        
        random_image = random.choice(last_20_percent)
        img_path = os.path.join(symbol_dir, random_image)
        
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Ошибка загрузки: {img_path}")
            return np.ones((45, 45), dtype=np.uint8) * 255
            
        return img
    
    def generate_sequence(self, length=10, include_spaces=True):
        """Генерирует случайную последовательность символов"""
        if include_spaces:
            # Генерируем последовательность с пробелами
            sequence = []
            for i in range(length):
                if i > 0 and i % 3 == 0:  # Добавляем пробел каждые 3 символа
                    sequence.append(' ')
                sequence.append(random.choice([c for c in self.available_symbols if c != ' ']))
            sequence = ''.join(sequence)
        else:
            sequence = ''.join(random.choices([c for c in self.available_symbols if c != ' '], k=length))
        
        print(f"Генерируем: '{sequence}'")
        images = []
        
        for char in sequence:
            img = self.get_random_symbol_image(char)
            if img is not None:
                images.append(img)
        
        return sequence, images

In [ ]:
def create_sequence_image(images, spacing=20):
    """Склеивает изображения символов в одну последовательность"""
    if not images:
        return np.ones((100, 100), dtype=np.uint8) * 255
    
    total_width = sum(img.shape[1] for img in images) + spacing * (len(images) - 1)
    max_height = max(img.shape[0] for img in images)
    
    sequence_img = np.ones((max_height, total_width), dtype=np.uint8) * 255
    
    x_offset = 0
    for img in images:
        h, w = img.shape
        y_offset = (max_height - h) // 2
        sequence_img[y_offset:y_offset+h, x_offset:x_offset+w] = img
        x_offset += w + spacing
    
    return sequence_img

# Тестируем генератор
generator = SymbolGenerator('C:\\improc\\hw9\\')
sequence, images = generator.generate_sequence(length=12, include_spaces=True)
sequence_image = create_sequence_image(images, spacing=15)

plt.figure(figsize=(12, 4))
plt.imshow(sequence_image, cmap='gray')
plt.title(f'Сгенерированная последовательность: "{sequence}"')
plt.axis('off')
plt.show()

In [ ]:
def preprocess_image(image):
    """Предобработка изображения для выделения контуров"""
    # Инвертируем изображение
    img_inverted = cv2.bitwise_not(image)
    
    # Применяем морфологические операции для улучшения качества
    kernel = np.ones((2, 2), np.uint8)
    img_processed = cv2.morphologyEx(img_inverted, cv2.MORPH_CLOSE, kernel)
    
    return img_processed

def extract_symbols(image):
    """Выделяет отдельные символы из изображения последовательности"""
    processed_img = preprocess_image(image)
    
    # Создаем цветное изображение для визуализации
    color_img = cv2.cvtColor(processed_img, cv2.COLOR_GRAY2BGR)
    
    # Находим контуры
    contours, _ = cv2.findContours(processed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Сортируем контуры слева направо
    contours_sorted = sorted(contours, key=lambda cnt: cv2.boundingRect(cnt)[0])
    
    symbols = []
    bounding_boxes = []
    
    for i, cnt in enumerate(contours_sorted):
        area = cv2.contourArea(cnt)
        x, y, w, h = cv2.boundingRect(cnt)
        
        # Фильтруем слишком маленькие области
        if w * h > 50 and w > 5 and h > 10:
            # Вырезаем символ
            symbol_img = image[y:y+h, x:x+w]
            
            # Нормализуем размер
            normalized_symbol = normalize_symbol_size(symbol_img, target_size=45)
            
            symbols.append((x, normalized_symbol))
            bounding_boxes.append((x, y, w, h))
            
            # Рисуем bounding box для визуализации
            cv2.rectangle(color_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    return symbols, bounding_boxes, color_img

def normalize_symbol_size(symbol_img, target_size=45):
    """Приводит символ к стандартному размеру"""
    h, w = symbol_img.shape
    
    # Создаем квадратное изображение
    squared = np.ones((target_size, target_size), dtype=np.uint8) * 255
    
    # Вычисляем смещения для центрирования
    y_offset = (target_size - h) // 2
    x_offset = (target_size - w) // 2
    
    # Вставляем символ в центр
    squared[y_offset:y_offset+h, x_offset:x_offset+w] = symbol_img
    
    return squared

In [ ]:
def load_training_data(data_path, symbols, train_size=0.8):
    """Загружает данные для обучения классификатора"""
    X = []
    y = []
    
    for symbol in symbols:
        folder_name = symbol
        if symbol == 'x':
            folder_name = 'X'
        elif symbol == '*':
            folder_name = 'times'
        elif symbol == ' ':
            continue  # Пропускаем пробелы для обучения
        
        symbol_path = os.path.join(data_path, folder_name)
        
        if not os.path.exists(symbol_path):
            print(f"Предупреждение: папка {symbol_path} не существует")
            continue
        
        images = [f for f in os.listdir(symbol_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
        images.sort()
        
        if not images:
            continue
        
        split_idx = int(len(images) * train_size)
        train_images = images[:split_idx]
        
        for img_name in train_images:
            img_path = os.path.join(symbol_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            if img is not None:
                # Нормализуем размер
                normalized_img = normalize_symbol_size(img, target_size=45)
                X.append(normalized_img.flatten())
                y.append(symbol)
    
    return np.array(X), np.array(y)

# Загружаем данные для обучения
symbols_for_training = ['0','1','2','3','4','5','6','7','8','9','+','-','t','x','y','(',')',',']
X_train, y_train = load_training_data('C:\\improc\\hw9\\', symbols_for_training, train_size=0.8)

print(f"Обучающая выборка: {len(X_train)} изображений")
print(f"Классы: {set(y_train)}")

In [ ]:
def train_knn_models(X_train, y_train):
    """Обучает несколько KNN моделей с разными k"""
    models = {}
    
    k_values = [1, 3, 5, 7]
    for k in k_values:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        models[f'knn_{k}'] = knn
        print(f"Модель KNN с k={k} обучена!")
    
    return models

# Обучаем модели
knn_models = train_knn_models(X_train, y_train)

In [ ]:
def predict_sequence(model, symbols_data):
    """Предсказывает последовательность символов"""
    predicted_symbols = []
    
    for x, symbol_img in symbols_data:
        # Подготавливаем изображение для классификации
        img_flat = symbol_img.flatten().reshape(1, -1)
        prediction = model.predict(img_flat)
        predicted_symbols.append(prediction[0])
    
    return ''.join(predicted_symbols)

def evaluate_model(model, symbols_data, true_sequence):
    """Оценивает точность модели"""
    predicted_sequence = predict_sequence(model, symbols_data)
    
    # Сравниваем с истинной последовательностью
    correct_chars = 0
    errors = []
    
    min_len = min(len(predicted_sequence), len(true_sequence))
    for i in range(min_len):
        if predicted_sequence[i] == true_sequence[i]:
            correct_chars += 1
        else:
            errors.append((i, true_sequence[i], predicted_sequence[i]))
    
    accuracy = correct_chars / len(true_sequence) if true_sequence else 0
    
    return predicted_sequence, accuracy, errors

In [ ]:
def recognize_handwritten_sequence(generated_sequence, generated_image, models):
    """Полный процесс распознавания рукописной последовательности"""
    print("=" * 60)
    print("ПРОЦЕСС РАСПОЗНАВАНИЯ РУКОПИСНОЙ ПОСЛЕДОВАТЕЛЬНОСТИ")
    print("=" * 60)
    
    # 1. Выделяем символы
    symbols_data, bounding_boxes, visualization = extract_symbols(generated_image)
    
    # Сортируем символы по координате X
    symbols_sorted = sorted(symbols_data, key=lambda x: x[0])
    
    print(f"Выделено символов: {len(symbols_sorted)}")
    print(f"Истинная последовательность: '{generated_sequence}'")
    
    # Визуализация выделенных символов
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(generated_image, cmap='gray')
    plt.title('Исходное изображение')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(visualization)
    plt.title('Выделенные контуры')
    plt.axis('off')
    
    # Показываем первые несколько выделенных символов
    plt.subplot(1, 3, 3)
    if symbols_sorted:
        sample_symbols = min(5, len(symbols_sorted))
        for i in range(sample_symbols):
            plt.subplot(1, sample_symbols, i+1)
            plt.imshow(symbols_sorted[i][1], cmap='gray')
            plt.title(f'Символ {i+1}')
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # 2. Классифицируем символы разными моделями
    results = {}
    
    for model_name, model in models.items():
        predicted_sequence, accuracy, errors = evaluate_model(model, symbols_sorted, generated_sequence)
        
        results[model_name] = {
            'predicted': predicted_sequence,
            'accuracy': accuracy,
            'errors': errors
        }
        
        print(f"\n--- {model_name.upper()} ---")
        print(f"Предсказано: '{predicted_sequence}'")
        print(f"Точность: {accuracy:.2%}")
        
        if errors:
            print("Ошибки:")
            for pos, true_char, pred_char in errors:
                print(f"  Позиция {pos}: ожидалось '{true_char}', получено '{pred_char}'")
    
    # 3. Выбираем лучшую модель
    best_model = max(results.items(), key=lambda x: x[1]['accuracy'])
    print(f"\n{'='*60}")
    print(f"ЛУЧШАЯ МОДЕЛЬ: {best_model[0]}")
    print(f"Точность: {best_model[1]['accuracy']:.2%}")
    print(f"Предсказание: '{best_model[1]['predicted']}'")
    print(f"Оригинал:    '{generated_sequence}'")
    print("=" * 60)
    
    return results, best_model

In [ ]:
# Генерируем тестовую последовательность
test_sequence, test_images = generator.generate_sequence(length=15, include_spaces=True)
test_image = create_sequence_image(test_images, spacing=20)

# Запускаем распознавание
results, best_model = recognize_handwritten_sequence(test_sequence, test_image, knn_models)

In [ ]:
def compare_all_models(results):
    """Сравнивает производительность всех моделей"""
    print("\n" + "="*70)
    print("СРАВНЕНИЕ ВСЕХ МОДЕЛЕЙ KNN")
    print("="*70)
    
    sorted_results = sorted(results.items(), key=lambda x: x[1]['accuracy'], reverse=True)
    
    print(f"{'Модель':<10} | {'Точность':<10} | {'Предсказание'}")
    print("-" * 70)
    
    for model_name, result in sorted_results:
        accuracy_percent = result['accuracy'] * 100
        print(f"{model_name:<10} | {accuracy_percent:>8.2f}% | '{result['predicted']}'")
    
    return sorted_results

# Сравниваем модели
comparison = compare_all_models(results)

In [ ]:
def run_multiple_tests(num_tests=3, sequence_length=10):
    """Запускает несколько тестов для оценки стабильности"""
    print("\n" + "="*70)
    print(f"ТЕСТИРОВАНИЕ НА {num_tests} ПРИМЕРАХ")
    print("="*70)
    
    all_accuracies = {model_name: [] for model_name in knn_models.keys()}
    
    for test_num in range(num_tests):
        print(f"\n--- Тест {test_num + 1} ---")
        
        # Генерируем последовательность
        sequence, images = generator.generate_sequence(length=sequence_length, include_spaces=True)
        sequence_image = create_sequence_image(images, spacing=15)
        
        # Выделяем символы
        symbols_data, _, _ = extract_symbols(sequence_image)
        symbols_sorted = sorted(symbols_data, key=lambda x: x[0])
        
        # Тестируем все модели
        for model_name, model in knn_models.items():
            predicted_sequence, accuracy, _ = evaluate_model(model, symbols_sorted, sequence)
            all_accuracies[model_name].append(accuracy)
            
            print(f"{model_name}: {accuracy:.2%} - '{predicted_sequence}'")
    
    # Выводим среднюю точность
    print(f"\n{'='*70}")
    print("СРЕДНЯЯ ТОЧНОСТЬ ПО ВСЕМ ТЕСТАМ:")
    print("="*70)
    
    for model_name, accuracies in all_accuracies.items():
        avg_accuracy = np.mean(accuracies) * 100
        std_accuracy = np.std(accuracies) * 100
        print(f"{model_name:<10} | Средняя: {avg_accuracy:>6.2f}% | Станд. отклонение: {std_accuracy:.2f}%")

# Запускаем множественное тестирование
run_multiple_tests(num_tests=3, sequence_length=12)